# DPPM: 离散目标检测过程中的预测过程

# 任务背景

基于深度学习的物体检测已成为人工智能领域的热门话题。在物体检测中，最常用的方法是设置默认框。设置默认方框的典型方法是根据多尺度结构形成多个特征图。不同的特征图可以预测尺度差异较大的默认方框参数，而同一特征图可以预测不同类型（尺度略有不同或长宽比不同）的默认方框参数。然而，在现有方法中，在同一个特征图上，不同类型的方框参数是在同一个过程中预测出来的。从感受野的角度来看，这一过程是不合理的，因为不同类型的默认方框需要不同的感受野。此外，从语义提取的角度来看，这也是不合适的，因为不同类型的默认方框在结构上有很大差异。

# 数据集

## 名称
The PASCAL Visual Object Classes

## 描述
The ​​PASCAL Object Recognition Database Collection​​ (PASCAL VOC) 是计算机视觉领域最具影响力的基准数据集之一，主要用于目标检测、分类和分割任务，包含20个常见物体类别（如人、动物、交通工具等），覆盖2005年至2012年的多版本数据，其中2012版包含11,530张训练/验证图像和27,450个标注实例，标注信息涵盖边界框、像素级分割及难度标签，其非标志性场景设计（如复杂街道中的目标）显著提升了模型泛化能力，并通过官方挑战赛推动算法评估标准化

## 来源
- 训练集/验证集（2007）<br>
链接：http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar<br>
图片样本数：5,011张（训练集2,501张 + 验证集2,510张）<br>
大小（tar文件）：450MB<br>
- 测试集（2007）<br>
链接：http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar<br>
图片样本数：4952张<br>
大小（tar文件）：430MB<br>  
- 训练集/验证集（2012）<br>
链接：http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar<br>
图片样本数：11,540张（训练集5,717张 + 验证集5,823张）<br>
大小（tar文件）：2GB<br>

## 样本举例
- 训练集/验证集（2007）：至少包含一只鸟的图片<br>
![至少包含一只鸟的图片](./images/bird.jpg "至少包含一只鸟的图片")

- 测试集（2007）：包含一只鸟的图片<br>
![包含一只鸟的图片](./images/bird_test.jpg "包含一只鸟的图片")

- 训练集/验证集（2012）：至少包含一个自行车的图片<br>
![至少包含一个自行车的图片](./images/bicycle.jpg "至少包含一个自行车的图片")


# 模型架构

DPPM结构的基本原理是在同一特征图中，不同类型的默认框参数预测过程被离散化。在此，假设特征图在每个位置对应四个默认框。设置四个默认框特征图的DPPM如图1所示，即DPPM4。<br>
![DPPM4](./images/fig1.png "DPPM4")
现有算法中用于输出默认框参数的特征图称为基础特征图。在现有方法中，基础特征图通过卷积过程直接输出四个默认框的参数，如图2所示，它们是共同训练的。<br>
![BASIC](./images/fig2.png "BASIC")
然而，本文在基础特征图之后引入了一个独立的学习过程，即DPPM。该DPPM将不同默认框的预测过程分离。如图2所示，每个实心箭头代表一次卷积，且不同类型默认框预测过程中的卷积核大小可配置。因此，可以为不同类型的框设置不同的卷积核。对于每个默认框的特征图，如map-box1、map-box2、map-box3和map-box4，其通道数为基础特征图大小的四分之一。此设置对应于可引入DPPM的对象检测算法。通过简单的拼接过程，可以获得相同的Loc（位置）和Conf（置信度）结构，如图1所示。无需更改离散过程输出的参数，即可方便地对应于原始过程。

# 模型实现和实验结果分析

## 训练（RFB300+DPPM）

In [7]:
run train_RFB.py

RFBNet(
  (base): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1,

/usr/local/lib/python3.8/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch:1 || epochiter: 0/517|| Totel iter 0 || L: 3.4107 C: 20.3990||Batch time: 4.8935 sec. ||LR: 0.00000100
Epoch:1 || epochiter: 10/517|| Totel iter 10 || L: 3.3056 C: 18.5881||Batch time: 0.2931 sec. ||LR: 0.00001647
Epoch:1 || epochiter: 20/517|| Totel iter 20 || L: 3.2797 C: 16.9292||Batch time: 0.2813 sec. ||LR: 0.00003194
Epoch:1 || epochiter: 30/517|| Totel iter 30 || L: 3.3275 C: 16.7212||Batch time: 0.2786 sec. ||LR: 0.00004741
Epoch:1 || epochiter: 40/517|| Totel iter 40 || L: 3.2061 C: 16.2536||Batch time: 0.2821 sec. ||LR: 0.00006288
Epoch:1 || epochiter: 50/517|| Totel iter 50 || L: 3.5699 C: 16.1959||Batch time: 0.2740 sec. ||LR: 0.00007835
Epoch:1 || epochiter: 60/517|| Totel iter 60 || L: 2.9899 C: 15.7355||Batch time: 0.2772 sec. ||LR: 0.00009382
Epoch:1 || epochiter: 70/517|| Totel iter 70 || L: 3.3175 C: 15.5687||Batch time: 0.2793 sec. ||LR: 0.00010929
Epoch:1 || epochiter: 80/517|| Totel iter 80 || L: 3.2048 C: 15.3512||Batch time: 0.2851 sec. ||LR: 0.00012476
Epo

## 测试（RFB300+DPPM）

In [5]:
run test_RFB.py

Finished loading model!
RFBNet(
  (base): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2

## 对比实验

In [6]:
cd ../RFBNet

/hy-tmp/RFBNet


## 训练（RFB300）

In [8]:
run train_RFB.py

RFBNet(
  (base): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1,

## 测试（RFB300）

In [10]:
run test_RFB.py

Finished loading model!
RFBNet(
  (base): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2

# 结果分析

表1展示了不同方法在PASCAL VOC数据集上的检测性能。<br>
<br>
### TABLE 1: Detection performance with the VOC 2007 test
| Method        | Backbone     | Data   | mAP/% |
|---------------|-------------|-------|-------|
| Faster [3]    | VGG-16      | 07+12 | 73.2  |
| Faster++ [26] | ResNet-101  | 07+12 | 76.4  |
| R-FCN [4]     | ResNet-101  | 07+12 | 80.5  |
| SSD300 [8]    | VGG-16      | 07+12 | 77.6  |
| DSSD321 [9]   | ResNet-101  | 07+12 | 78.6  |
| RFB300 [10]   | VGG-16      | 07+12 | 79.8  |
| SSD300+DPPM   | VGG-16      | 07+12 | 79.0  |
| RFB300+DPPM   | VGG-16      | 07+12 | 76.3  |
<br>
当将DPPM引入SSD时，其准确率达到了79.0%mAP。与原始准确率77.6%mAP相比，这一提升值得注意。与DSSD（78.6%）及其他算法相比，SSD+DPPM方法也实现了更高的准确率。因此，与现有将小目标置于深度预测中的方法相比，SSD+DPPM方法无疑更为简洁有效。当将DPPM引入RFB时，由于PASCAL VOC数据集的评估标准相对不足且目标相对简单，准确率仅有轻微下降。RFB+DPPM方法的有效性基于COCO验证标准得到验证。该比较结果如图1所示。<br>
![ANALYSIS](./images/fig3.png "ANALYSIS")
如图1所示，当阈值在0.5至0.75范围内时，添加DPPM后的结构mAP略低于原始算法。然而，在0.75至0.95范围内，添加DPPM后的mAP得到显著提升。不难得出结论，在某些情况下，RFB+DPPM预测的性能优于RFB。
